# Baseline model

## Import modules and load data

In [10]:
import cv2
import numpy as np
import os
import pandas as pd
import pickle

In [11]:
data_path = os.path.join(os.getcwd(), '..', 'data')
labels = os.path.join(data_path, 'traininglabels.csv')

In [12]:
df = pd.read_csv(labels)
df.head()

,image_id,has_oilpalm,score
0,img_000002017.jpg,0,0.7895
1,img_000012017.jpg,0,1.0000
2,img_000022017.jpg,0,1.0000
3,img_000072017.jpg,0,1.0000
4,img_000082017.jpg,0,1.0000


In [13]:
height = 256
width = 256

In [14]:
def img_as_array(image_id, size=None):
    image_path = os.path.join(data_path, 'train_images', image_id)
    img = cv2.imread(str(image_path))
    if size is None:
        return img
    return cv2.resize(img, size)

In [15]:
images = []
responses = []
scores = []
for idx, row in df.iterrows():
    img = img_as_array(row[0])  #, size=(height, width))
    images.append(img.reshape(1, height, width, 3))
    responses.append(row[1])
    scores.append(row[2])

In [16]:
images = np.concatenate(images, axis=0)
responses = np.array(responses).reshape(images.shape[0], 1)
scores = np.array(scores).reshape(images.shape[0], 1)

In [17]:
print("responses: {}".format(responses.shape))
print("scores: {}".format(scores.shape))
print("images: {}".format(images.shape))

responses: (15244, 1)
scores: (15244, 1)
images: (15244, 256, 256, 3)


## Pickle train set for future usage

*NOTE: Images aren't normalized!*

In [18]:
with open(os.path.join(data_path, 'train_images_256x256.pkl'), 'wb') as fout:
    pickle.dump(images, fout, protocol=4)
with open(os.path.join(data_path, 'train_responses.pkl'), 'wb') as fout:
    pickle.dump(responses, fout, protocol=4)
with open(os.path.join(data_path, 'train_scores.pkl'), 'wb') as fout:
    pickle.dump(scores, fout, protocol=4)

In [19]:
images = images / 255.

## Define model architecture

In [11]:
from keras.applications import vgg16
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential, Model
from keras.optimizers import SGD

Using TensorFlow backend.


In [12]:
def baseline_vgg():
    vgg = vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(height, width,3), pooling='max')
    last = vgg.output
    # x = Flatten()(last)
    x = Dense(128, activation='relu')(last)
    x = Dropout(0.5)(x)
    x = Dense(1, activation='sigmoid')(x)
    return Model(inputs=[vgg.input], outputs=[x])

model = baseline_vgg()
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=1e-4, momentum=0.9), metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [13]:
model.fit(images, responses, batch_size=32, epochs=10)

Epoch 1/10
15244/15244 [==============================] - 114s 7ms/step - loss: 0.1057 - acc: 0.9619
Epoch 2/10
15244/15244 [==============================] - 111s 7ms/step - loss: 0.0404 - acc: 0.9876
Epoch 3/10
15244/15244 [==============================] - 111s 7ms/step - loss: 0.0323 - acc: 0.9904
Epoch 4/10
15244/15244 [==============================] - 109s 7ms/step - loss: 0.0283 - acc: 0.9900
Epoch 5/10
15244/15244 [==============================] - 109s 7ms/step - loss: 0.0273 - acc: 0.9913
Epoch 6/10
15244/15244 [==============================] - 109s 7ms/step - loss: 0.0247 - acc: 0.9919
Epoch 7/10
15244/15244 [==============================] - 109s 7ms/step - loss: 0.0219 - acc: 0.9931
Epoch 8/10
15244/15244 [==============================] - 109s 7ms/step - loss: 0.0184 - acc: 0.9939
Epoch 9/10
15244/15244 [==============================] - 109s 7ms/step - loss: 0.0174 - acc: 0.9934
Epoch 10/10
15244/15244 [==============================] - 109s 7ms/step - loss: 0.0176 - a

In [14]:
model.save('vgg_baseline_10epoch.h5')